## Setup

**Import Statements**

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep
from sklearn.metrics import r2_score
from amb_sdk.sdk import DarwinSdk

**Connection to Darwin**

In [2]:
# Login
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')
status, msg = ds.auth_login_user('mprost@utexas.edu', 'FbSnZ663sf')

if not status:
    print(msg)


**Setup Path**

In [3]:
path = './sets/'

## Upload and Read Data

**Read Data**

In [31]:
dataset_name = 'remittances_data_q2.csv'
df = pd.read_csv(os.path.join(path, dataset_name))
df.head()

/Users/Matt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,period,source_code,source_name,source_region,source_income,source_lending,source_G8G20,destination_code,destination_name,...,cc2 fx margin,cc2 total cost %,inter lcu bank fx,transparent,Standard Note,note2,receiving network coverage,pickup method,date,corridor
0,620160001,2016_2Q,AGO,Angola,Sub-Saharan Africa,Lower middle income,IBRD,..,NAM,Namibia,...,7.49,14.36,0.09,yes,NaN,NaN,High,Agent,11/May/2016,AGONAM
1,620160002,2016_2Q,AGO,Angola,Sub-Saharan Africa,Lower middle income,IBRD,..,NAM,Namibia,...,4.19,11.78,0.09,yes,NaN,NaN,High,Agent,11/May/2016,AGONAM
2,620160003,2016_2Q,AGO,Angola,Sub-Saharan Africa,Lower middle income,IBRD,..,NAM,Namibia,...,0.00,10.93,1.00,no,This RSP is not transparent: the exchange rate...,NaN,Low,NaN,11/May/2016,AGONAM
3,620160004,2016_2Q,ARE,United Arab Emirates,Middle East & North Africa,High income,..,..,BGD,Bangladesh,...,0.08,1.17,21.34,yes,NaN,NaN,High,Agent,09/May/2016,AREBGD
4,620160005,2016_2Q,ARE,United Arab Emirates,Middle East & North Africa,High income,..,..,BGD,Bangladesh,...,0.13,0.95,21.34,yes,NaN,NaN,High,Agent,09/May/2016,AREBGD


**Upload to Darwin**

In [69]:
# Upload dataset
status, message = ds.delete_dataset(dataset_name)
status, dataset = ds.upload_dataset(os.path.join(path, dataset_name))
if not status:
    print(dataset)

## Clean Data

In [70]:
# clean dataset
target = "cc1 total cost %"
status, job_id = ds.clean_data(dataset_name, target=target)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)
    


{'status': 'Requested', 'starttime': '2019-04-20T23:35:00.984645', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['remittances_data_q2.csv'], 'artifact_names': ['bfc8efe6948941ca93d2680d842b092e'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-20T23:35:00.984645', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['remittances_data_q2.csv'], 'artifact_names': ['bfc8efe6948941ca93d2680d842b092e'], 'model_name': None, 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-20T23:35:00.984645', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['remittances_data_q2.csv'], 'artifact_names': ['bfc8efe6948941ca93d2680d842b092e'], 'model_name': None, 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-04-20T23:35:00.984645', 'endti

## Create and Train Model

In [58]:
model = target + "_model0"
ds.delete_model(model)
status, job_id = ds.create_model(dataset_names = dataset_name, \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-20T23:17:17.524073', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['remittances_data_q2.csv'], 'artifact_names': None, 'model_name': 'cc1 total cost %_model0', 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-20T23:17:17.524073', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['remittances_data_q2.csv'], 'artifact_names': None, 'model_name': 'cc1 total cost %_model0', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-20T23:17:17.524073', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['remittances_data_q2.csv'], 'artifact_names': None, 'model_name': 'cc1 total cost %_model0', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-20T23:17:17.524073', 'endtime': None, 'percent_complete': 0, 'job_type': 'Train

In [67]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])

{'status': 'Running', 'starttime': '2019-04-20T23:33:32.464342', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzeModel', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['bc05bcc450ae42a88cd05f53cf2b7f4e'], 'model_name': 'cc1 total cost %_model0', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-04-20T23:33:32.464342', 'endtime': '2019-04-20T23:33:42.207201', 'percent_complete': 100, 'job_type': 'AnalyzeModel', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['bc05bcc450ae42a88cd05f53cf2b7f4e'], 'model_name': 'cc1 total cost %_model0', 'job_error': ''}


In [62]:
feature_importance[:10]

cc2 total cost %                          0.803116
cc1 lcu fee                               0.074618
firm_type = Money Transfer Operator       0.051828
cc2 lcu fee                               0.011829
cc2 fx margin                             0.010887
cc1 fx margin                             0.009801
cc2 lcu amount                            0.006971
access point = Bank branch,Call Center    0.003915
cc1 lcu amount                            0.003798
speed actual = Less than one hour         0.002624
dtype: float64

## Analyze Data

**Model Prediction**

In [73]:
print (dataset_name)
print (model)
status, artifact = ds.run_model(dataset_name, model)
print (status)
print (artifact)
sleep(1)
ds.wait_for_job(artifact['job_name'])
'''print (artifact)
status, artifact1 = ds.run_model(dataset_name, model)

#sleep(1)
#if status:
#    ds.wait_for_job(artifact['job_name'])
#else:
#    print(artifact)

#ds.wait_for_job(artifact['job_name'])
print (status)
print (artifact1)'''

remittances_data_q2.csv
cc1 total cost %_model0
False
400: Bad Request - <html>
<head><title>400 Bad Request</title></head>
<body>
<center><h1>400 Bad Request</h1></center>
<hr><center>nginx</center>
</body>
</html>



TypeError: string indices must be integers

**Get Predictions from Darwin**

In [52]:
status, prediction = ds.download_artifact(artifact['artifact_name'])
prediction.head()

TypeError: string indices must be integers

In [ ]:
#Plot predictions vs actual
print(len(prediction[target]))
#print(prediction[target][0])
prediction[target] = prediction[target].replace('Less than one hour', 0.05)
df[target] = df[target].replace('Less than one hour', 0.05)
prediction[target] = prediction[target].replace('Same day', 1)
df[target] = df[target].replace('Same day', 1)
prediction[target] = prediction[target].replace('Next day', 2)
df[target] = df[target].replace('Next day', 2)
prediction[target] = prediction[target].replace('2 days', 3)
df[target] = df[target].replace('2 days', 3)
prediction[target] = prediction[target].replace('3-5 days', 4)
df[target] = df[target].replace('3-5 days', 4)
prediction[target] = prediction[target].replace('6 days or more', 7)
df[target] = df[target].replace('6 days or more', 7)
#print(prediction[target][0])
#prediction[target].replace("Less than one hour", 0.05)
#prediction[target].replace("Less than one hour", 0.05)
#for i in range(0, 4574):
#    if prediction[target][i] == "Less than one hour":
#        prediction[target][i] = 0.05
plt.plot(df[target], prediction[target], '.')
plt.plot([0,2.3e7],[0,2.3e7],'--k')
plt.show()
print('R^2 : ', r2_score(df[target], prediction[target]))